In [18]:
import os
import pandas as pd
import numpy as np

# read the files in the folder preprocessed_datasets
path = 'preprocessed_datasets'
files = os.listdir(path)
files = [file for file in files if file.endswith('.csv')]
print(files)

NO3_train = pd.read_csv('preprocessed_datasets/NO3_train.csv')
NO3_val = pd.read_csv('preprocessed_datasets/NO3_val.csv')
NO3_test = pd.read_csv('preprocessed_datasets/NO3_test.csv')

print(f"n NO3_train: {NO3_train.head()}")

['NO1_test.csv', 'NO1_train.csv', 'NO1_val.csv', 'NO2_test.csv', 'NO2_train.csv', 'NO2_val.csv', 'NO3_test.csv', 'NO3_train.csv', 'NO3_val.csv', 'NO4_test.csv', 'NO4_train.csv', 'NO4_val.csv', 'NO5_test.csv', 'NO5_train.csv', 'NO5_val.csv']
n NO3_train:                    timestamp  consumption  temperature  time_of_day  \
0  2017-05-02 00:00:00+00:00     0.227701     0.450877     0.000000   
1  2017-05-02 01:00:00+00:00     0.222187     0.450877     0.043478   
2  2017-05-02 02:00:00+00:00     0.226947     0.443860     0.086957   
3  2017-05-02 03:00:00+00:00     0.236823     0.431579     0.130435   
4  2017-05-02 04:00:00+00:00     0.287226     0.426316     0.173913   

   time_of_week  time_of_year  lag_1_hour  lag_24_hours  
0      0.166667      0.331507    0.236276      0.246874  
1      0.166667      0.331507    0.227701      0.248087  
2      0.166667      0.331507    0.222187      0.248142  
3      0.166667      0.331507    0.226947      0.250252  
4      0.166667      0.331507

In [19]:
NO3_train = NO3_train.drop('timestamp', axis=1)
NO3_val = NO3_val.drop('timestamp', axis=1)
NO3_test = NO3_test.drop('timestamp', axis=1)

# Separate features and target for the training set
NO3_train_features = NO3_train.drop('consumption', axis=1).to_numpy(dtype=np.float32)
NO3_train_targets = NO3_train['consumption'].to_numpy(dtype=np.float32)

# Separate features and target for the validation set
NO3_val_features = NO3_val.drop('consumption', axis=1).to_numpy(dtype=np.float32)
NO3_val_targets = NO3_val['consumption'].to_numpy(dtype=np.float32)

# Separate features and target for the test set
NO3_test_features = NO3_test.drop('consumption', axis=1).to_numpy(dtype=np.float32)
NO3_test_targets = NO3_test['consumption'].to_numpy(dtype=np.float32)

print(f"n NO3_train: {NO3_train.head()}")
print(f"NO3_train.shape: {NO3_train.shape}")

n NO3_train:    consumption  temperature  time_of_day  time_of_week  time_of_year  \
0     0.227701     0.450877     0.000000      0.166667      0.331507   
1     0.222187     0.450877     0.043478      0.166667      0.331507   
2     0.226947     0.443860     0.086957      0.166667      0.331507   
3     0.236823     0.431579     0.130435      0.166667      0.331507   
4     0.287226     0.426316     0.173913      0.166667      0.331507   

   lag_1_hour  lag_24_hours  
0    0.236276      0.246874  
1    0.227701      0.248087  
2    0.222187      0.248142  
3    0.226947      0.250252  
4    0.236823      0.268047  
NO3_train.shape: (40920, 7)


In [24]:
# Create a nn with TENSORFLOW to train on the train set, validate on the validation set and test on the test set
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

# Create the neural network
model = keras.Sequential([
    layers.Dense(10, activation='relu', input_shape=[6]),
    layers.Dense(10, activation='relu'),
    layers.Dense(1)
])

# Compile the model
model.compile(optimizer='sgd', loss='mean_squared_error')

# Train the model
model.fit(NO3_train_features, NO3_train_targets, epochs=5, validation_data=(NO3_val_features, NO3_val_targets))

# Validate the model
val_loss = model.evaluate(NO3_val_features, NO3_val_targets)
print(f"Validation loss: {val_loss}")

# Test the model
test_loss = model.evaluate(NO3_test_features, NO3_test_targets)
print(f"Test loss: {test_loss}")

# save the model
model.save('my_model.keras')
print("Model saved")

2.16.0-rc0
Epoch 1/5
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 1s 495us/step - loss: 0.0195 - val_loss: 0.0010
Epoch 2/5
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 1s 443us/step - loss: 9.5963e-04 - val_loss: 6.7383e-04
Epoch 3/5
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 1s 448us/step - loss: 6.9873e-04 - val_loss: 5.7882e-04
Epoch 4/5
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 1s 522us/step - loss: 6.0096e-04 - val_loss: 5.1446e-04
Epoch 5/5
1279/1279 ━━━━━━━━━━━━━━━━━━━━ 1s 503us/step - loss: 5.4421e-04 - val_loss: 4.7727e-04
274/274 ━━━━━━━━━━━━━━━━━━━━ 0s 414us/step - loss: 5.9560e-04
Validation loss: 0.00047726885532028973
274/274 ━━━━━━━━━━━━━━━━━━━━ 0s 304us/step - loss: 5.6524e-04
Test loss: 0.00047388943494297564
Model saved
